<a href="https://colab.research.google.com/github/liorhameir/ml-projects/blob/master/Qlearning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [148]:
import numpy as np
import gym
import random
import time
from IPython.display import clear_output
from tqdm.notebook import tqdm

In [211]:
env = gym.make("FrozenLake8x8-v0")

# [up, down, right, left]
action_space_size = env.action_space.n
# [board size]
state_space_size = env.observation_space.n

q_table = np.zeros((state_space_size, action_space_size))

max_steps_per_episode = 300

learning_rate = 0.1
discount_rate = 0.999

max_exploration_rate = 1
min_exploration_rate = 0.01
exploration_decay_rate = 0.0001
exploration_rate = 1
sofar_episodes = 0


def train(num_episodes, watch=False):
    global exploration_rate
    global sofar_episodes
    global q_table
    rewards_all_episodes = np.zeros(num_episodes)
    
    for episode in tqdm(range(num_episodes)):
        state = env.reset()
        rewards_current_episode = 0
        for step in range(1, max_steps_per_episode + 1):
            if watch:
                clear_output(wait=True)
                env.render()
                time.sleep(0.3)
            exploration_rate_threshold = random.uniform(0, 1)
            if exploration_rate_threshold > exploration_rate:
                action = np.argmax(q_table[state, :])
            else:
                action = env.action_space.sample()
            new_state, reward, done, info = env.step(action)
            rewards_current_episode += reward
            q_table[state, action] = ((q_table[state, action] * (1 - learning_rate)) + \
                                     learning_rate * (reward + (discount_rate * np.max(q_table[new_state, :]))))
            state = new_state
            if done:
                if watch:
                    clear_output(wait=True)
                    env.render()
                    if reward == 1:
                        q_table[state, action] = 1
                        print("win +1")
                        time.sleep(3)
                    else:
                        print("loss")
                        time.sleep(3)
                break
            # dynamic programing

        exploration_rate = min_exploration_rate + (max_exploration_rate - min_exploration_rate) * np.exp(
            -exploration_decay_rate * (episode + sofar_episodes))
        rewards_all_episodes[episode] = rewards_current_episode
        

    sofar_episodes += num_episodes
    if watch:
        env.close()
        print("success rate:")
        print(rewards_all_episodes.sum() / num_episodes)
    else:
        rewards_per_thousand_episodes = np.split(rewards_all_episodes, num_episodes / 10000)
        count = 10000


        for r in rewards_per_thousand_episodes:
            print(count, ": ", str(sum(r / 10000)))
            count += 10000


train(100000, watch=False)






10000 :  0.04010000000000012
20000 :  0.27679999999998584
30000 :  0.5615999999999545
40000 :  0.7207999999999369
50000 :  0.7916999999999291
60000 :  0.8121999999999269
70000 :  0.8255999999999254
80000 :  0.8318999999999247
90000 :  0.8274999999999252
100000 :  0.8312999999999248


In [215]:
learning_rate = 0.01
train(10000, watch=False)


10000 :  0.8533999999999223


In [216]:
train(5, watch=True)

  (Right)
SFFFFFFF
FFFFFFFF
FFFHFFFF
FFFFFHFF
FFFHFFFF
FHHFFFHF
FHFFHFHF
FFFHFFFG
win +1

success rate:
1.0
